In [ ]:
import numpy as np

#my own convex hull
def myConvexHull(bucket):
    solution = np.array([[0.0,0.0]])
    sort = sorted(bucket,key=lambda x:(x[0],x[1]))

    #create p1 dan pn
    solution[0] = sort[0]
    solution = np.vstack((solution, sort[len(sort)-1]))
    hulSim = np.array([[5, 5]])
    hulSim = np.delete(hulSim, 0, axis=0)
    s1 = np.array([[1.0, 2.0]])
    s1 = np.delete(s1, 0, axis=0)
    s2 = np.array([[1.0, 2.0]])
    s2 = np.delete(s2, 0, axis=0)
    
    #membagi menjadi s1 dan s2
    for i  in range (len(bucket)):
        if((bucket[i,0] != solution[0,0] or bucket[i,1] != solution[0,1]) or (bucket[i,0] != solution[1,0] or bucket[i,1] != solution[1,1])):
            if((findDeter(solution[0,0], solution[0,1], solution[1,0], solution[1,1], bucket[i,0], bucket[i,1]) > 0)):  
                if((abs(findDeter(solution[0,0], solution[0,1], solution[1,0], solution[1,1], bucket[i,0], bucket[i,1]))) > 1e-12):
                    s1 = np.vstack((s1, bucket[i]))
            else:
                if((abs(findDeter(solution[0,0], solution[0,1], solution[1,0], solution[1,1], bucket[i,0], bucket[i,1]))) > 1e-12):
                    s2 = np.vstack((s2, bucket[i]))

    hulSim = np.append(hulSim, np.array(findHull(bucket, hulSim, solution, s1, solution[0], solution[1])), axis=0)
    hulSim = np.append(hulSim, np.array(findHull(bucket, hulSim,solution, s2, solution[1], solution[0])), axis=0)

    return hulSim

def findHull(bucket, hulSim,solution, S,A,B):
    if(S.size == 0):
        temp = np.array([[0, 0]])
        temp[0][0] = findIndex(bucket, A[0], A[1])
        temp[0][1] = findIndex(bucket, B[0], B[1])
        return temp
    else:
        solution = np.array([[1.0, 2.0]])
        solution = np.delete(solution, 0, axis=0)
        fartest = S[0]
        tempDistance = 0.00
        # Find Orthogonally farthest point from AB
        for i in range (len(S)):
            d = np.linalg.norm(np.cross(B-A, A-S[i]))/np.linalg.norm(B-A)

            if(d > tempDistance):
                tempDistance = d
                fartest = S[i]

        x1 = np.array([[1.0, 2.0]])
        x1 = np.delete(x1, 0, axis=0)
        x2 = np.array([[1.0, 2.0]])
        x2 = np.delete(x2, 0, axis=0) 

        #membuang titik dalam segitiga & membagi x1 dan x2
        for i  in range (len(S)):
            if (pointInTriangle(A[0], A[1], B[0], B[1], fartest[0], fartest[1], S[i,0], S[i,1]) == False) :
                if(findDeter(A[0], A[1], fartest[0], fartest[1], S[i,0], S[i,1]) > 0):
                    x1 = np.vstack((x1, S[i]))
                else:
                    x2 = np.vstack((x2, S[i]))

        solution = np.append(solution, np.array(findHull(bucket, hulSim,solution, x1, A, fartest)), axis=0)
        solution = np.append(solution, np.array(findHull(bucket, hulSim,solution,x2,fartest,B)), axis=0)

        return solution

def pointInTriangle(x1, y1, x2, y2, x3, y3, x, y):
    #output true untuk titik yang berada dalam segitiga
    denominator = ((y2 - y3)*(x1 - x3) + (x3 - x2)*(y1 - y3))
    a = ((y2 - y3)*(x - x3) + (x3 - x2)*(y - y3)) / denominator
    b = ((y3 - y1)*(x - x3) + (x1 - x3)*(y - y3)) / denominator
    c = 1 - a - b
    return 0 <= a and a <= 1 and 0 <= b and b <= 1 and 0 <= c and c <= 1

def findDeter(a,b,c,d,x,y):
    # output deterimanan dari matriks buatan
    matriks = np.array([[a, b, 1], [c, d, 1], [0,0,1]])    
    matriks[2,0] = x
    matriks[2,1] = y
    d = np.linalg.det(matriks)
    return d

def findIndex(bucket, x, y):
    #mencari indeks dari titik untuk membuat garis
    for i in range (len(bucket)):
        if(bucket[i][0] == x and bucket[i][1] == y):
            return i

In [ ]:
#dataset iris
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets

data = datasets.load_iris()

df = pd.DataFrame(data.data, columns=data.feature_names)
df['Target'] = pd.DataFrame(data.target)
df.head()

In [ ]:
#visualisasi hasil ConvexHull dataset iris: Sepal length vs sepal width
plt.figure(figsize = (10, 6))
atribut1 = 0        #ubah nilai atribut2 dengan atribut yang diinginkan
atribut2 = 1        #ubah nilai atribut2 dengan atribut yang diinginkan
colors = ['b','r','g']
plt.title(data.feature_names[atribut1] + ' vs ' + data.feature_names[atribut2])
plt.xlabel(data.feature_names[atribut1])
plt.ylabel(data.feature_names[atribut2])
for i in range(len(data.target_names)):
    bucket = df[df['Target'] == i]
    bucket = bucket.iloc[:,[atribut1,atribut2]].values
    hullMonic = myConvexHull(bucket)
    hullMonic = hullMonic.astype(int)
    plt.scatter(bucket[:, 0], bucket[:, 1], label=data.target_names[i])
    for simplexMonic in hullMonic:
        plt.plot(bucket[simplexMonic, 0], bucket[simplexMonic, 1], colors[i])
plt.legend()

#visualisasi hasil ConvexHull dataset iris: petal length vs petal width
plt.figure(figsize = (10, 6))
atribut1 = 2        #ubah nilai atribut2 dengan atribut yang diinginkan
atribut2 = 3        #ubah nilai atribut2 dengan atribut yang diinginkan
colors = ['b','r','g']
plt.title(data.feature_names[atribut1] + ' vs ' + data.feature_names[atribut2])
plt.xlabel(data.feature_names[atribut1])
plt.ylabel(data.feature_names[atribut2])
for i in range(len(data.target_names)):
    bucket = df[df['Target'] == i]
    bucket = bucket.iloc[:,[atribut1,atribut2]].values
    hullMonic = myConvexHull(bucket)
    hullMonic = hullMonic.astype(int)
    plt.scatter(bucket[:, 0], bucket[:, 1], label=data.target_names[i])
    for simplexMonic in hullMonic:
        plt.plot(bucket[simplexMonic, 0], bucket[simplexMonic, 1], colors[i])
plt.legend()

In [ ]:
#dataset wine
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets

data = datasets.load_wine()

df = pd.DataFrame(data.data, columns=data.feature_names)
df['Target'] = pd.DataFrame(data.target)
df.head()

In [ ]:
#visualisasi hasil ConvexHull dataset wine: Sepal Malic Acid vs Alcohol
plt.figure(figsize = (10, 6))
atribut1 = 0        #ubah nilai atribut2 dengan atribut yang diinginkan
atribut2 = 1        #ubah nilai atribut2 dengan atribut yang diinginkan
colors = ['b','r','g']
plt.title(data.feature_names[atribut1] + ' vs ' + data.feature_names[atribut2])
plt.xlabel(data.feature_names[atribut1])
plt.ylabel(data.feature_names[atribut2])
for i in range(len(data.target_names)):
    bucket = df[df['Target'] == i]
    bucket = bucket.iloc[:,[atribut1,atribut2]].values
    hullMonic = myConvexHull(bucket)
    hullMonic = hullMonic.astype(int)
    plt.scatter(bucket[:, 0], bucket[:, 1], label=data.target_names[i])
    for simplexMonic in hullMonic:
        plt.plot(bucket[simplexMonic, 0], bucket[simplexMonic, 1], colors[i])
plt.legend()

#visualisasi hasil ConvexHull dataset wine: Ash vs Alcalinity of Ash
plt.figure(figsize = (10, 6))
atribut1 = 2        #ubah nilai atribut2 dengan atribut yang diinginkan
atribut2 = 3        #ubah nilai atribut2 dengan atribut yang diinginkan
colors = ['b','r','g']
plt.title(data.feature_names[atribut1] + ' vs ' + data.feature_names[atribut2])
plt.xlabel(data.feature_names[atribut1])
plt.ylabel(data.feature_names[atribut2])
for i in range(len(data.target_names)):
    bucket = df[df['Target'] == i]
    bucket = bucket.iloc[:,[atribut1,atribut2]].values
    hullMonic = myConvexHull(bucket)
    hullMonic = hullMonic.astype(int)
    plt.scatter(bucket[:, 0], bucket[:, 1], label=data.target_names[i])
    for simplexMonic in hullMonic:
        plt.plot(bucket[simplexMonic, 0], bucket[simplexMonic, 1], colors[i])
plt.legend()

In [ ]:
#dataset breast_cancer
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets

data = datasets.load_breast_cancer()

df = pd.DataFrame(data.data, columns=data.feature_names)
df['Target'] = pd.DataFrame(data.target)
df.head()

In [ ]:
#visualisasi hasil ConvexHull dataset breast cancer: mean radius vs mean texture
plt.figure(figsize = (10, 6))
atribut1 = 0        #ubah nilai atribut1 dengan atribut yang diinginkan
atribut2 = 1        #ubah nilai atribut2 dengan atribut yang diinginkan
colors = ['b','r','g']
plt.title(data.feature_names[atribut1] + ' vs ' + data.feature_names[atribut2])
plt.xlabel(data.feature_names[atribut1])
plt.ylabel(data.feature_names[atribut2])
for i in range(len(data.target_names)):
    bucket = df[df['Target'] == i]
    bucket = bucket.iloc[:,[atribut1,atribut2]].values
    hullMonic = myConvexHull(bucket)
    hullMonic = hullMonic.astype(int)
    plt.scatter(bucket[:, 0], bucket[:, 1], label=data.target_names[i])
    for simplexMonic in hullMonic:
        plt.plot(bucket[simplexMonic, 0], bucket[simplexMonic, 1], colors[i])
plt.legend()

#visualisasi hasil ConvexHull dataset breast cancer: mean area vs mean compactness
plt.figure(figsize = (10, 6))
atribut1 = 3        #ubah nilai atribut1 dengan atribut yang diinginkan
atribut2 = 5        #ubah nilai atribut2 dengan atribut yang diinginkan
colors = ['b','r','g']
plt.title(data.feature_names[atribut1] + ' vs ' + data.feature_names[atribut2])
plt.xlabel(data.feature_names[atribut1])
plt.ylabel(data.feature_names[atribut2])
for i in range(len(data.target_names)):
    bucket = df[df['Target'] == i]
    bucket = bucket.iloc[:,[atribut1,atribut2]].values
    hullMonic = myConvexHull(bucket)
    hullMonic = hullMonic.astype(int)
    plt.scatter(bucket[:, 0], bucket[:, 1], label=data.target_names[i])
    for simplexMonic in hullMonic:
        plt.plot(bucket[simplexMonic, 0], bucket[simplexMonic, 1], colors[i])
plt.legend()